# HSCT survival linear regression model

## Notebook set-up

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import uniform
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV

## 1. Load data

In [ ]:
# Load the datasets
dictionary_df=pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv')
training_df=pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
testing_df=pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')

# Save the ID and drop
testing_ids=testing_df['ID']
training_df.drop('ID', axis=1, inplace=True)
testing_df.drop('ID', axis=1, inplace=True)

# Remove the labels from the training dataframe
training_labels=training_df['efs_time']
training_df.drop(['efs', 'efs_time'], axis=1, inplace=True)

print(f'Training features: {training_df.shape}')
print(f'Testing features: {testing_df.shape}')

## 2. Split categorical and numerical features

In [ ]:
# Get lists of categorical and numerical column names
categorical_feature_names=dictionary_df['variable'][dictionary_df['type'] == 'Categorical']
numerical_feature_names=dictionary_df['variable'][dictionary_df['type'] == 'Numerical']

# Remove the feature column from the column names lists
categorical_feature_names=categorical_feature_names[categorical_feature_names != 'efs']
numerical_feature_names=numerical_feature_names[numerical_feature_names != 'efs_time']

# Split the training and testing dataframes
training_categorical_df=training_df[categorical_feature_names].copy()
training_numerical_df=training_df[numerical_feature_names].copy()
testing_categorical_df=testing_df[categorical_feature_names].copy()
testing_numerical_df=testing_df[numerical_feature_names].copy()

print(f'Training numerical features: {training_numerical_df.shape}')
print(f'Training categorical features: {training_categorical_df.shape}')
print(f'Testing numerical features: {testing_numerical_df.shape}')
print(f'Testing categorical features: {testing_categorical_df.shape}')

## 3. Handle missing data

### 3.1. Categorical features

In [ ]:
training_categorical_df.fillna('Missing', inplace=True)
testing_categorical_df.fillna('Missing', inplace=True)

print(f'Training categorical features: {training_categorical_df.shape}')
print(f'Testing categorical features: {testing_categorical_df.shape}')

### 3.2. Numerical features

In [ ]:
imputer=KNNImputer(n_neighbors=3, weights='uniform')
imputer.fit(training_numerical_df)

training_numerical_data=imputer.transform(training_numerical_df)
testing_numerical_data=imputer.transform(testing_numerical_df)

training_numerical_df=pd.DataFrame(training_numerical_data, columns=training_numerical_df.columns)
testing_numerical_df=pd.DataFrame(testing_numerical_data, columns=training_numerical_df.columns)

print(f'Training numerical features: {training_numerical_df.shape}')
print(f'Testing numerical features: {testing_numerical_df.shape}')

## 3. One-hot encode categorical features

In [ ]:
encoder=OneHotEncoder(drop='first',sparse_output=False)
encoder.fit(training_categorical_df)
feature_names=encoder.get_feature_names_out()

training_categorical_data=encoder.transform(training_categorical_df)
testing_categorical_data=encoder.transform(testing_categorical_df)

training_categorical_df=pd.DataFrame(training_categorical_data, columns=feature_names)
testing_categorical_df=pd.DataFrame(testing_categorical_data, columns=feature_names)

training_features_df=pd.concat(
    [
        training_numerical_df.reset_index(drop=True), 
        training_categorical_df.reset_index(drop=True)
    ],
    axis=1
)

testing_features_df=pd.concat(
    [
        testing_numerical_df.reset_index(drop=True), 
        testing_categorical_df.reset_index(drop=True)
    ],
    axis=1
)

print(f'Training features: {training_features_df.shape}')
print(f'Testing features: {testing_features_df.shape}')

## 4. Hyperparameter tuning

In [ ]:
# Instantiate the model
model=ElasticNet(max_iter=50000, random_state=315)

# Define the cross-validation strategy
cross_validation=KFold(n_splits=7, shuffle=True, random_state=315)

# Define the hyperparameter search space
distributions={
    'alpha': uniform(loc=0, scale=100),
    'l1_ratio': uniform(),
    'fit_intercept': [True, False],
    'positive': [True, False],
    'selection': ['cyclic', 'random']
}

# Set-up the search
search=RandomizedSearchCV(
    model,
    distributions,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    cv=cross_validation,
    n_iter=1000,
    random_state=315,
    return_train_score=True
)

results=search.fit(training_features_df, training_labels)
print(f'Best hyperparameters: {results.best_params_}')

## 5. Fit and evaluate tuned model

In [ ]:
# Instantiate the model with the tuned hyperparameters
model=ElasticNet(**results.best_params_, max_iter=50000, random_state=315)
baseline_scores=cross_val(model, training_features, training_labels)

model.fit(training_features, training_labels)
predicted_efs_time=model.predict(training_features)

# Set-up a 2x1 figure
fig, axs=plt.subplots(1,2, figsize=(9,4))
axs=axs.flatten()

fig.suptitle('Linear regression model performance')

axs[0].set_title('Actual vs predicted EFS time')
axs[0].scatter(training_labels, predicted_efs_time, color='black')
axs[0].set_xlabel('True efs time')
axs[0].set_ylabel('Predicted EFS time')

axs[1].set_title('EFS prediction error')
axs[1].hist(training_labels-predicted_efs_time, density=True, bins=30, color='black')
axs[1].set_xlabel('RMSE')
axs[1].set_ylabel('Probability')

plt.tight_layout()
plt.show()

## 6. Make test set predictions

In [ ]:
predicted_efs_time=model.predict(testing_features_df)
scaler=MinMaxScaler()
scaler.fit(predicted_efs_time.reshape(-1, 1))
predicted_efs_time=scaler.transform(predicted_efs_time.reshape(-1, 1))

predictions_df=pd.DataFrame.from_dict(
    {
        'ID': testing_ids,
        'prediction': predicted_efs_time.flatten()
    }
)

predictions_df.head()